In [1]:
#Reading news data
import pandas as pd
import numpy as np

data = {
    'amazon': 
        { 
            'news_url': 'https://news.yahoo.com/amazon-pushing-hard-ocean-shipping-200001052.html',
            'historical_news_path': 'news_data/amazon_news.csv',
            'similarities': [0.7791, 0.5986, 0.7309, 0.7061, 0.7184, 0.648, 0.6707, 0.4732, 0.7404, 0.7302, 0.7908, 0.7849, 0.7267, 0.7621, 0.7944, 0.7392, 0.5898]
        },
    'tradewar':{
            'news_url':'https://www.nytimes.com/live/2021/03/17/business/stock-market-today',
            'historical_news_path':'news_data/tradewar.csv',
            'similarities':[0.5961, 0.5891, 0.2861, 0.6685, 0.6292, 0.7172]
    },
    'fedhikerates':{
            'news_url':'https://www.cnbc.com/2018/12/19/stock-markets-dow-futures-edge-higher-federal-reserve-rate-decision.html',
            'historical_news_path':'news_data/fedhikerates.csv',
            'similarities':[0.7096, 0.7773, 0.4398, 0.6639, 0.6728, 0.7551, 0.7779]
    }
} 

use_cached_data = True;
data_type = 'tradewar'
token = '3e609e553c274103bf2acd8dd9029d8c'


#url of the news to be analysed
news_url = data[data_type]['news_url']

In [2]:
#Sentiment analysis of the news
import requests

url = 'https://api.dandelion.eu/datatxt/sent/v1/?lang=en&token=' + token + '&url=' + news_url
response = requests.get(url)
res_json = response.json()
if 'sentiment' in res_json:
    print("Sentiment of the news article:", res_json['sentiment'])

Sentiment of the news article: {'type': 'negative', 'score': -0.555921052631579}


In [3]:
#Read historical news

news_data = pd.read_csv(data[data_type]['historical_news_path'],sep=";")
news_data['Date'] = pd.to_datetime(news_data['Date']).dt.date
# news_data.to_csv('news_data/fedhikerates.csv',sep=";",index=False,columns=news_data.columns)
news_data.head()


,Source,Date,Title,Link
0,The New York Times,2021-10-04,U.S. Signals No Thaw in Trade Relations With C...,https://www.nytimes.com/2021/10/04/business/ec...
1,Reuters,2021-01-14,"U.S.-China trade war has cost up to 245,000 U....",https://www.reuters.com/article/us-usa-trade-c...
2,Washington Post,2021-09-22,U.S. tariffs on Chinese goods didn’t bring com...,https://www.washingtonpost.com/politics/2021/0...
3,Fortune,2020-11-08,Why a Biden presidency won’t end the U.S.-Chin...,https://fortune.com/2020/11/09/joe-biden-us-ch...
4,Time,2019-08-07,The U.S.-China Trade War Is Steering the World...,https://time.com/5645964/donald-trump-china-tr...


In [4]:
# Find similarities of current news and historical news
similarities = []

if not use_cached_data:
    for url in news_data['Link']:
        url = 'https://api.dandelion.eu/datatxt/sim/v1/?url1=' + news_url + '&url2=' + url + '&token=' + token
        response = requests.get(url)
        json_data = response.json()
    #     print(data)
        if('similarity' in json_data):
            similarities.append(json_data['similarity'])
else:
    similarities = data[data_type]['similarities']
        
print(similarities)
print("Mean of similarities: ", np.mean(similarities))

[0.5961, 0.5891, 0.2861, 0.6685, 0.6292, 0.7172]
Mean of similarities:  0.5810333333333334


In [5]:
#Get the stock data for the date range of the news

stock_data = pd.read_csv('stock_data/spy500_historical_data.csv', sep=',')
stock_data.rename(columns={'Close/Last':'Close'}, inplace=True)
stock_data['Change_Stock'] = stock_data['Close'].diff(periods=-1)
stock_data['Date'] = pd.to_datetime(stock_data['Date']).dt.date
stock_data = stock_data.drop(columns=['Volume', 'Open', 'High' ,'Low'])

stock_data.head()

,Date,Close,Change_Stock
0,2021-11-23,468.19,0.62
1,2021-11-22,467.57,-1.32
2,2021-11-19,468.89,-0.84
3,2021-11-18,469.73,1.59
4,2021-11-17,468.14,-1.14


In [6]:
#Plot a graph of history with changes as per news

merged_data = pd.merge(stock_data, news_data, on = "Date" )
merged_data.drop_duplicates(subset="Date", keep='last', inplace=True)
merged_data.set_index('Date', inplace=True)
# merged_data['Change_Stock'].loc[(merged_data['Change_Stock'] > 0)] = -1
merged_data


,Close,Change_Stock,Source,Title,Link
Date,,,,,
2021-10-04,428.64,-5.60,The New York Times,U.S. Signals No Thaw in Trade Relations With C...,https://www.nytimes.com/2021/10/04/business/ec...
2021-09-22,431.86,-1.77,Washington Post,U.S. tariffs on Chinese goods didn’t bring com...,https://www.washingtonpost.com/politics/2021/0...
2021-01-14,378.46,-1.33,Reuters,"U.S.-China trade war has cost up to 245,000 U....",https://www.reuters.com/article/us-usa-trade-c...
2019-08-07,284.97,-2.83,Time,The U.S.-China Trade War Is Steering the World...,https://time.com/5645964/donald-trump-china-tr...
2019-05-13,280.86,-7.24,The Indian Express,"The US-China trade war, and its impact on India",https://indianexpress.com/article/explained/si...
2018-06-01,273.60,2.66,The New Yorker,Trump’s New Tariffs Are Causing America’s Clos...,https://www.newyorker.com/news/our-columnists/...


In [7]:
from datetime import datetime, timedelta
import plotly.graph_objs as go
from chart_studio import plotly as py
import cufflinks as cf
cf.go_offline()
from plotly.offline import download_plotlyjs, init_notebook_mode, plot,iplot


dateset= list(sorted(merged_data.index))

changelist = []
for date in dateset:
    try:
        changelist.append(merged_data.loc[date]['Change_Stock'].round(3))
    except:
        changelist.append(0)

trace_close = go.Scatter(
                x=stock_data["Date"],
                y=stock_data['Close'],
                name = "Close")

colorlist = []
for s in changelist:
    if s > 0:
        colorlist.append('green')
    elif s == 0:
        colorlist.append('rgb(150, 150, 150)')
    else:
        colorlist.append('red')
        
shapes = [dict(
            x0 = s,
            x1 = s + timedelta(days=1),
            y0 = 0,
            y1 = 1,
            xref= 'x',
            yref = 'paper',
            opacity =  0.3,
            line = {'color': colorlist[i], 'width': 5}
            ) for (i,s) in enumerate(dateset)]


annotations =  [dict(
            x = s,
            y = (0.05 + i*0.2) % 1,
            xref = 'x',
            yref = 'paper',
            showarrow = False,
            xanchor = 'left',
            ax = 20,
            ay = -30,
            bordercolor = '#000000',
            borderwidth = 2,
            borderpad = 4,
            bgcolor = colorlist[i],
            font = {
                'family':'Courier New, monospace',
                'size':16,
                'color':'#ffffff'
            },
            opacity = 0.8,
            text =  changelist[i]
            
            ) for (i,s) in enumerate(dateset)]


fig_data = [trace_close]

layout = dict(
    title = "Stock prices",
    shapes = shapes,
    annotations = annotations,
    xaxis = dict(range = [merged_data.index.min() - pd.DateOffset(months=2),merged_data.index.max() + pd.DateOffset(months=2)]),
    yaxis = dict(title = 'Close price')
)


fig = dict(data=fig_data, layout=layout)
go.Figure(fig).show()

In [8]:
# Calculate t-test

"""
https://www.statisticshowto.com/probability-and-statistics/t-test/
How big is “big enough”? 
Every t-value has a p-value to go with it. 
A p-value is the probability that the results from your sample data occurred by chance. 
For example, a p value of 5% is 0.05. Low p-values are good; They indicate your data did not occur by chance. 
For example, a p-value of .01 means there is only a 1% probability that the results from an experiment happened by chance. 
In most cases, a p-value of 0.05 (5%) is accepted to mean the data is valid.
"""

from scipy import stats

historical_news_dates =  list(sorted(merged_data.index))

min_date = min(historical_news_dates) - pd.DateOffset(months=2)
max_date = max(historical_news_dates) + pd.DateOffset(months=2)
relevant_stock_data = stock_data[(stock_data["Date"] >= min_date)  & (stock_data["Date"] <= max_date)]
relevant_stock_data.set_index('Date', inplace=True)
relevant_stock_dates = list(relevant_stock_data.index)


remaining_dates = set(relevant_stock_dates) - set(historical_news_dates)

# print(len(relevant_stock_dates), len(historical_news_dates), len(remaining_dates))

b = list(relevant_stock_data.loc[historical_news_dates]["Change_Stock"])
a = list(relevant_stock_data.loc[remaining_dates]["Change_Stock"])

_, p2 = stats.ttest_ind(a,b,equal_var = True)
impact_score = (10 - p2*10).round(1)
sentiment = res_json['sentiment']['type'].capitalize()

print("t-statistics =", p2)
print("Impact score =", impact_score)
print("Sentiment = ", sentiment)

t-statistics = 0.06723115690255665
Impact score = 9.3
Sentiment =  Negative


/home/sandesh/.local/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning:

Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.

